In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle
import matplotlib.pyplot as plt

In [2]:
from process import map_pred

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
TEMP = OUTPUT + 'temp/'
MAP  = OUTPUT + 'map/'

In [4]:
# original input
df_train = pd.read_csv(FOLDER+'train.csv')
df_test = pd.read_csv(FOLDER+'test.csv')

In [5]:
DATA_DATE = '20190721'
VER = '02'
MAP_DATE = '20190728'
MODEL_DATE = '20190804'
num_rep = 2

In [6]:
# input files
# pred_train = OUTPUT + DATA_DATE + '_' + 'predict_train.pickle'
# pred_test =  OUTPUT + DATA_DATE + '_' + 'submission.csv'

pred_train = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'pred_train_{}.pickle'.format(num_rep)
pred_test = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'submission_{}.csv'.format(num_rep)

map_train = MAP + MAP_DATE + '_' + 'map_train.pickle'
map_test = MAP + MAP_DATE + '_' + 'map_test.pickle'

dist_train = MAP + MAP_DATE + '_' + 'dist_train.pickle'
dist_test = MAP + MAP_DATE + '_' + 'dist_test.pickle'

In [7]:
# output files
temp_no = 0
num_div = 4

pred_dist_train_temp = TEMP + 'pred_dist_train_{}.pickle'
pred_dist_test_temp = TEMP + 'pred_dist_test_{}.pickle'

pred_dist_train = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'pred_dist_train.pickle'
pred_dist_test = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'pred_dist_test.pickle'

In [8]:
df_pred_train = pd.read_pickle(pred_train)
df_pred_train = df_pred_train.rename(columns={'scalar_coupling_constant':'predict'})
df_pred_train_idx = pd.merge(df_train, df_pred_train, on=['id']).set_index('molecule_name')
df_pred_train_idx.head()

,id,atom_index_0,atom_index_1,type,scalar_coupling_constant,predict
molecule_name,,,,,,
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,85.010650
dsgdb9nsd_000001,1,1,2,2JHH,-11.2570,-11.223590
dsgdb9nsd_000001,2,1,3,2JHH,-11.2548,-11.299028
dsgdb9nsd_000001,3,1,4,2JHH,-11.2543,-11.299028
dsgdb9nsd_000001,4,2,0,1JHC,84.8074,85.010650


In [9]:
df_pred_train_idx.tail()

,id,atom_index_0,atom_index_1,type,scalar_coupling_constant,predict
molecule_name,,,,,,
dsgdb9nsd_133884,4658142,17,4,2JHC,3.543450,0.507619
dsgdb9nsd_133884,4658143,17,5,3JHC,0.568997,-0.316933
dsgdb9nsd_133884,4658144,17,6,3JHC,1.173370,-0.299367
dsgdb9nsd_133884,4658145,17,7,2JHC,4.762010,0.520170
dsgdb9nsd_133884,4658146,17,8,1JHC,117.934000,79.953299


In [10]:
mols_train = df_train['molecule_name'].unique()

In [11]:
df_map_train_idx = pd.read_pickle(map_train).set_index('molecule_name')
df_dist_train_idx = pd.read_pickle(dist_train).set_index('molecule_name')

In [13]:
start = time.time()
df_map_pred_train = pd.DataFrame()

div = len(mols_train) // num_div

# for m in mols_train[:10]:
for m in mols_train[div*temp_no:div*(temp_no+1)]:
    if m == mols_train[2]:
        continue
    df_map_pred_train_temp = map_pred(df_pred_train_idx, df_map_train_idx, m)
    df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0)
    
if temp_no == 0:
    m = mols_train[2]
    se_temp = pd.Series()
#     se_temp['molecule_name']  = m
    se_temp['0'] = df_pred_train_idx.loc[m]['predict']
    for i in range(48):
        se_temp['{}'.format(i+1)] = 0.0
    cols = df_map_pred_train_temp.columns[:-1]
    df_map_pred_train_temp = pd.DataFrame(se_temp).T
    df_map_pred_train_temp.columns = cols
    df_map_pred_train_temp['id'] = df_pred_train_idx.loc[m]['id']
    df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0, sort=False)

df_pred_dist_train = pd.merge(df_map_pred_train, df_dist_train_idx, on=['id'])
df_pred_dist_train.to_pickle(pred_dist_train_temp.format(temp_no))

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

total elapsed_time:1700.918690443039[sec]


In [14]:
sum(df_pred_dist_train['0_x'].isnull())

0

In [15]:
df_pred_dist_train.shape

(958409, 99)

In [16]:
df_pred_test = pd.read_csv(pred_test)
df_pred_test = df_pred_test.rename(columns={'scalar_coupling_constant':'predict'})
df_pred_test_idx = pd.merge(df_test, df_pred_test, on=['id']).set_index('molecule_name')

In [17]:
mols_test = df_test['molecule_name'].unique()

In [18]:
df_map_test_idx = pd.read_pickle(map_test).set_index('molecule_name')
df_dist_test_idx = pd.read_pickle(dist_test).set_index('molecule_name')

In [19]:
start = time.time()
df_map_pred_test = pd.DataFrame()

div = len(mols_test) // num_div
# for m in mols_test[:10]:
for m in mols_test[div*temp_no:div*(temp_no+1)]:
    df_map_pred_test_temp = map_pred(df_pred_test_idx, df_map_test_idx, m)
    df_map_pred_test = pd.concat([df_map_pred_test, df_map_pred_test_temp], axis=0, sort=False)

df_pred_dist_test = pd.merge(df_map_pred_test, df_dist_test_idx, on=['id'])
df_pred_dist_test.to_pickle(pred_dist_test_temp.format(temp_no))

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

total elapsed_time:520.9655029773712[sec]


In [20]:
df_pred_dist_test.shape

(516698, 99)

In [21]:
sum(df_pred_dist_test['0_x'].isnull())

0

In [24]:
df_pred_dist_train_list = []
df_pred_dist_test_list = []

for i in range(num_div):
    df_pred_dist_train_list.append(pd.read_pickle(pred_dist_train_temp.format(i)))
    df_pred_dist_test_list.append(pd.read_pickle(pred_dist_test_temp.format(i)))
    
df_pred_dist_train = pd.concat(df_pred_dist_train_list, axis=0,sort=False)
df_pred_dist_test = pd.concat(df_pred_dist_test_list, axis=0,sort=False)

df_pred_dist_train.to_pickle(pred_dist_train)
df_pred_dist_test.to_pickle(pred_dist_test)

print(df_pred_dist_train.shape, df_pred_dist_test.shape)

(4658147, 99) (2505542, 99)


In [23]:
# (4658147, 100) (2505542, 100)